In [10]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json

In [11]:
def parse_socratic_parsing(explanation):
    prompt = PromptTemplate.from_template("""You are a world class text analyzer that can help in parsing an given
    INPUT. INPUT is provided after ### Your job is to parse the total number of questions that were answered "Yes"
    and the total number of questions present. Return the output in a JSON format with following keys. The
    description of each key below is given next to it. - socratic_explanations - List containing all the answers to
    the questions - socratic_results - List containing values "Yes" or "No". Each element in the list corresponds to
    one question in the input. "Yes" is choosen when the question was answered Yes else No.

    Place the above presented JSON formatted answer in the <output></output> tags.
    Look at the following examples on how the output has to be formatted.

    Example 1:
    INPUT:
    1. Does the answer mention any specific circumstances under which an employee can self-register a username using a foreign telephone number?
    - Yes, The answer mentions that Flex allows an employee to register with a foreign telephone number if they also use a foreign address.

    2. Does the answer state that the user's account setup must match exactly with the foreign address and phone number in order to register using a foreign telephone number?
    - No, The answer does not mention anything about the user's account setup needing to match exactly with the foreign address and phone number.

    3. Does the answer mention it is possible to self-register with a foreign telephone number?
    - Yes, The answer mentions that Flex allows an employee to register with a foreign telephone number.

    Total Number of Questions: 3
    Total questions answered with Yes: 2

    OUTPUT: <output>{{"socratic_explanations": ["Yes, The answer mentions that Flex allows an employee to register with a foreign telephone number if they also use a foreign address.",
    "No, The answer does not mention anything about the user's account setup needing to match exactly with the foreign address and phone number.",
    "Yes, The answer mentions that Flex allows an employee to register with a foreign telephone number."],
     "socratic_results": ["Yes", "No", "Yes"]}}</output>


    Example 2:
    INPUT:
    1. Does the answer mention about use of Splunk queries to search for data in a specific index?
    - The answer does not mention anything about using Splunk queries to search for data in a specific index.

    2. Does the answer explain how to specify which fields to include in the query result set?
    - The answer does not explain how to specify which fields to include in the query result set.

    3. Does the answer provide an example of a basic Splunk query and explain how it works?
    - The answer does not provide an example of a basic Splunk query and explain how it works.

    4. Does the answer contain a basic Splunk query?
    - The answer does not contain a basic Splunk query.

    OUTPUT: <output>{{"socratic_explanations": ["The answer does not mention anything about using Splunk queries to search for data in a specific index.",
    "The answer does not explain how to specify which fields to include in the query result set.",
    "The answer does not provide an example of a basic Splunk query and explain how it works.",
    "The answer does not contain a basic Splunk query."], "socratic_results": ["No", "No", "No", "No"]}}</output>

    Example 3:
    INPUT:
    1. Does the answer mention about Configuration > Payroll Policies > Timecard Pay Policies > Create New Policy as the location to set up a Flex Time payroll policy?
    - NO

    2. Does the answer mention about employees must have worked for at least 90 days before they are eligible for holiday pay?
    - YES

    3. Does the answer mention about the Qualifying Period can be set to 90 days?
    - YES

    4. Does the answer mention about the policy will only pay eligible employees for holidays based on qualifying criteria?
    - YES

    5. Does the answer mention about the policy will automatically be applied to all employees in the company?
    - NO

    Total Number of Questions: 5
    Total questions answered with Yes: 3

    OUTPUT: <output>{{"socratic_explanations": ["No, The answer does not mention about Configuration > Payroll Policies > Timecard Pay Policies > Create New Policy as the location to set up a Flex Time payroll policy?",
    "Yes, the answer does mention about employees must have worked for at least 90 days before they are eligible for holiday pay.",
    "Yes, the answer mentions about the Qualifying Period can be set to 90 days",
    "Yes,  the answer mention about the policy will only pay eligible employees for holidays based on qualifying criteria",
    "No, the answer does not mention about the policy will automatically be applied to all employees in the company"],
    "socratic_results": ["No", "Yes", "Yes", "Yes", "No"]}}</output>

    Example 4:
    INPUT:
    1. Does the answer mention that US Department of Labor website provides information about minimum wage requirements in a particular state?
    - Yes, the answer mentions that you can find information about minimum wage requirements in a particular state by visiting the U.S. Department of Labor's Wage and Hour Division website.

    2. Does the answer mention that latest minimum wage requirements for each state should be determined by using external websites?
    - No, the answer does not mention that the latest minimum wage requirements for each state should be determined by using external websites.

    3. Does the answer mention about cities or counties have higher minimum wage rates than the state minimum wage requirements?
    - No, the answer does not mention about cities or counties having higher minimum wage rates than the state minimum wage requirements.

    OUTPUT: <output>{{"socratic_explanations": ["Yes, the answer mentions that you can find information about minimum wage requirements in a particular state by visiting the U.S. Department of Labor's Wage and Hour Division website.",
    "No, the answer does not mention that the latest minimum wage requirements for each state should be determined by using external websites.",
    "No, the answer does not mention about cities or counties having higher minimum wage rates than the state minimum wage requirements."], "socratic_results": ["Yes", "No", "No"]}}</output>

    Now read the below INPUT and respond accordingly.
    ###
    INPUT: {input}
    OUTPUT:

    """,)
    model = OpenAI(api_key="", temperature=0)
    output = model(prompt.format(input=explanation))
    return output


In [39]:
prompt = """
You are a world-class researcher that tests presence of required facts in an answer
            You are assessing a submitted answer on a given task based on a criterion. Here is the data:

            Answer the questions given after [BEGIN DATA] tags from the [CONTEXT] provided after [BEGIN DATA].
            Answer in YES or NO only. Provide only the answers as a bullet points of "YES" or "NO".
            Please repeat each question that was asked followed by "YES" or "NO" answer.
            "YES" followed by the explanation would be given in the cases where a question can be answered from the CONTEXT.
            "NO" followed by the explanation in the cases where the question cannot be answered. Ignore the upper and lower case in answer.

            Check the following examples on how a set of Questions are answered.
            After looking at the examples, look at the Questions after ### and CONTEXT after ### to respond.
            Add "Label - None" at the end.

            [EXAMPLE 1]
            [Questions]:
            1. Does the answer mention that Paychex Insurance Agency offers general liability insurance to its clients?
            2. Does the answer provide a citation for the information mentioned in the context?
            3. Does the answer mention the presence of key elements in the context, such as general liability insurance and Paychex Insurance Agency?
            ************
            [CONTEXT]:
            Yes, Paychex Insurance Agency offers general liability insurance to its clients. 1 Citations: 1. 161109165611730
            ************
            Answer:
            1. Does the answer mention that Paychex Insurance Agency offers general liability insurance to its clients?
            Yes, the answer mentions that Paychex Insurance Agency offers general liability insurance to its clients
            2. Does the answer provide a citation for the information mentioned in the context?
            Yes, the answers provides a citation for the information mentioned in the context.
            3. Does the answer mention the presence of key elements in the context, such as general liability insurance and Paychex Insurance Agency?
            Yes, the answers mentions the presence of key elements in the context, such as general liability insurance and Paychex Insurance Agency.

            Total Number of Questions: 3
            Total questions answered with Yes: 3
            Label - None
            ************

            [EXAMPLE 2]
            [Questions]:
            1. Does the answer mention that Paychex Insurance Agency offers general liability insurance to its clients?
            2. Does the answer provide a citation for the information mentioned in the context?
            ************
            [CONTEXT]:
            Yes, Paychex Insurance Agency offers general liability insurance
            ************
            Answer:
            1. Does the answer mention that Paychex Insurance Agency offers general liability insurance to its clients?
            Yes, the answer mentions that Paychex Insurance Agency offers general liability insurance to its clients.
            2. Does the answer provide a citation for the information mentioned in the context?
            No, the answer does not provide a citation for the information mentioned in the context.

            Total number of Questions: 2
            Total questions answered with Yes: 1
            Label - None
            ************

            Now answer each of the questions from [Questions] given below from the CONTEXT given below.

            ###
            ************
            [BEGIN DATA]
            ************
            [Questions]: {criterion}
            ************
            [CONTEXT]: {generated_answer}
            ************
            Answer:
            ************
            [END DATA]
"""

In [42]:
generated_answer= ''' 
    Valid for 12 months from the first visit
    Free General Parking (with entry until 6pm, not valid for Valet or Preferred parking, not valid on black-out dates)
    Invitation to Special Events
    Discounted Halloween Horror Night Tickets
    15% In-Park Food, Beverage & Merchandise Discount
    Discount on General Admission Tickets (discounts vary per season, available online and in-park, limit of 9 discounted tickets per transaction)
    10-15% Discount at Participating CityWalk Locations
    
    Please note that blackout dates apply on return visits, and the pass does not include discounts on in-Park food, beverage, or merchandise.'''
criterion= '''
        
        1. Does the response mention valid for 12 months from the first visit?
        2. Does the response mention parking with entry up to 6pm?
        3. Does the response mention a 15 percent in park discount on food, beverage & merchandise?
        4. Does the response mention sweepstakes opportunities?
        
        
'''

In [49]:
def compute_socratic_score(criterion: str, generated_reponse: str, ):
    chat = ChatOpenAI(temperature=0, openai_api_key="",)
    human_message_prompt = HumanMessagePromptTemplate.from_template(prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
    [human_message_prompt]
)

    # get a chat completion from the formatted messages
    llm_response= chat(
        chat_prompt.format_prompt(
        generated_answer= generated_reponse, criterion= criterion
        ).to_messages()
    )
    look= parse_socratic_parsing(llm_response.content)
    output_parsed = look.split("<output>")[-1].split("</output>")
    response_dict = json.loads(output_parsed[0])['socratic_results']
    return response_dict

In [50]:
compute_socratic_score(criterion= criterion, generated_reponse= generated_answer)

['YES', 'YES', 'YES', 'NO']